In [1]:
!pip install openai==0.28.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
!pip install sumy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 20.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21692 sha256=1c5797e7a65cb776aaffe494a5333752a2882960c80a97fe175c4e090812a03d
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=bb4212078a49ebd786e0973a0e676769128f81ef1cbbd5822358e11ce3290273
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=1068

In [90]:
import pandas as pd
import re
import requests
import os
import openai
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
from collections import Counter
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
import pickle


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Assuming you have set your API key and endpoint as environment variables
openai.api_key = os.getenv("b8e6ac2cfda244dd848a823511255a0b")
openai.api_base = os.getenv("https://hackathonservice.openai.azure.com/")

deployment_name = 'gpt-3.5-turbo'  # Ensure this matches your deployment name in Azure

In [3]:
from google.colab import files
uploaded = files.upload()  # This will prompt you to upload files from your computer


KeyboardInterrupt: ignored

In [6]:
from google.colab import files
uploaded = files.upload()  # This will prompt you to upload files from your computer



Saving Company Profile.txt to Company Profile.txt
Saving Job Descriptions.txt to Job Descriptions.txt
Saving Meeting Notes.txt to Meeting Notes.txt
Saving Project Documentation.txt to Project Documentation.txt
Saving Team Structure.txt to Team Structure.txt


In [7]:
from google.colab import files
uploaded = files.upload()

Saving Company Profile.txt to Company Profile (1).txt
Saving Job Descriptions.txt to Job Descriptions (1).txt
Saving Meeting Notes.txt to Meeting Notes (1).txt
Saving System Documentation.txt to System Documentation.txt
Saving Team Structure.txt to Team Structure (1).txt


In [8]:
from google.colab import files
uploaded = files.upload()

Saving Company Profile.txt to Company Profile (2).txt
Saving Job Descriptions.txt to Job Descriptions (2).txt
Saving Meeting Notes.txt to Meeting Notes (2).txt
Saving Project Documentation.txt to Project Documentation (1).txt
Saving Team Structure.txt to Team Structure (2).txt


In [4]:
# Read SOPs file
with open('SOPs.txt', 'r') as file:
    sops = file.read()

# Read the first set of company files
with open('Company Profile.txt', 'r') as file:
    company_profile = file.read()
with open('Job Descriptions.txt', 'r') as file:
    job_descriptions = file.read()
with open('Meeting Notes.txt', 'r') as file:
    meeting_notes = file.read()
with open('Project Documentation.txt', 'r') as file:
    project_documentation = file.read()
with open('Team Structure.txt', 'r') as file:
    team_structure = file.read()

# Read the second set of company files (renamed by Google Colab)
with open('Company Profile (1).txt', 'r') as file:
    company_profile_1 = file.read()
with open('Job Descriptions (1).txt', 'r') as file:
    job_descriptions_1 = file.read()
with open('Meeting Notes (1).txt', 'r') as file:
    meeting_notes_1 = file.read()
with open('System Documentation.txt', 'r') as file:
    system_documentation = file.read()  # This seems to be a different file, not a second version
with open('Team Structure (1).txt', 'r') as file:
    team_structure_1 = file.read()

# Read the third set of company files
with open('Company Profile (2).txt', 'r') as file:
    company_profile_2 = file.read()
with open('Job Descriptions (2).txt', 'r') as file:
    job_descriptions_2 = file.read()
with open('Meeting Notes (2).txt', 'r') as file:
    meeting_notes_2 = file.read()
with open('Project Documentation (1).txt', 'r') as file:
    project_documentation_1 = file.read()  # This seems to be a second version of project documentation
with open('Team Structure (2).txt', 'r') as file:
    team_structure_2 = file.read()


In [5]:
def parse_sops(text):
    # Split the text into different SOPs based on the number and title pattern
    sops = re.split(r'\d+\.\s+[A-Za-z ]+\n', text)

    # The first element is often empty due to the split, so remove it if it is
    if sops[0].strip() == '':
        sops = sops[1:]

    # Extract titles using regex
    titles = re.findall(r'\d+\.\s+([A-Za-z ]+)', text)

    # Check if the number of SOPs matches the number of titles
    if len(sops) != len(titles):
        raise ValueError("Mismatch between number of SOPs and titles")

    # Process each SOP
    data = []
    for title, sop in zip(titles, sops):
        # Split each SOP into steps
        steps = sop.strip().split('\n')
        data.append({"Title": title, "Steps": steps})

    # Create a DataFrame from the parsed data
    return pd.DataFrame(data)

def parse_company_profile(text):
    lines = text.strip().split('\n')
    data = [line.split(':', 1) for line in lines if ':' in line]  # Ensure only lines with ':' are included
    return pd.DataFrame(data, columns=['Category', 'Information']).applymap(lambda x: x.strip())

def parse_job_descriptions(text):
    # Split the text into job descriptions by double newlines
    jobs = text.strip().split('\n\n')
    data = []
    for job in jobs:
        sections = job.strip().split('\n')
        title = sections[0].strip()

        # Initialize indices to -1 (not found)
        responsibilities_index = -1
        requirements_index = -1

        # Try to find the index of 'Responsibilities' and 'Requirements'
        for i, section in enumerate(sections):
            if 'Responsibilities:' in section:
                responsibilities_index = i + 1
            elif 'Requirements:' in section:
                requirements_index = i + 1
                break  # Stop searching once we find 'Requirements'

        # If both sections were found, extract the content
        if responsibilities_index != -1 and requirements_index != -1:
            responsibilities = ' '.join(sections[responsibilities_index:requirements_index-1]).strip('Responsibilities:').strip()
            requirements = ' '.join(sections[requirements_index:]).strip('Requirements:').strip()
            data.append([title, responsibilities, requirements])

    # Create a DataFrame from the parsed data
    df = pd.DataFrame(data, columns=["Position", "Responsibilities", "Requirements"])
    return df

# Rest of your code for fetching the file and printing the DataFrame...

def parse_project_documentation(text):
    # Pattern to identify section headers
    pattern = r'##\s*(.+)'

    # Split the text into sections based on the pattern
    sections = re.split(pattern, text)

    # The first split is often empty, so remove it if it is
    if sections[0].strip() == '':
        sections = sections[1:]

    # Pair each header with its corresponding content
    data = []
    for i in range(0, len(sections), 2):
        # Check for out-of-bounds
        if i+1 < len(sections):
            section_title = sections[i].strip()
            section_content = sections[i+1].strip()
            data.append([section_title, section_content])

    # Create a DataFrame from the parsed data
    return pd.DataFrame(data, columns=["Section", "Content"])


In [7]:
# Parse the files using your functions
parsed_sops = parse_sops(sops)

parsed_company_profile = parse_company_profile(company_profile)
parsed_job_descriptions = parse_job_descriptions(job_descriptions)
parsed_meeting_notes = parse_company_profile(meeting_notes) # Assuming the same format as company profile
parsed_project_documentation = parse_project_documentation(project_documentation)
parsed_team_structure = parse_company_profile(team_structure)

parsed_company_profile_1 = parse_company_profile(company_profile_1)
parsed_job_descriptions_1 = parse_job_descriptions(job_descriptions_1)
parsed_meeting_notes_1 = parse_company_profile(meeting_notes_1)
parsed_system_documentation = parse_project_documentation(system_documentation) # Assuming similar format to project documentation
parsed_team_structure_1 = parse_company_profile(team_structure_1)

parsed_company_profile_2 = parse_company_profile(company_profile_2)
parsed_job_descriptions_2 = parse_job_descriptions(job_descriptions_2)
parsed_meeting_notes_2 = parse_company_profile(meeting_notes_2)
parsed_project_documentation_1 = parse_project_documentation(project_documentation_1)
parsed_team_structure_2 = parse_company_profile(team_structure_2)

In [8]:
parsed_sops

,Title,Steps
0,Client Acquisition SOP,[Identify potential clients through market res...
1,Project Planning SOP,"[Once a client contract is signed, assign a pr..."
2,Client Communication SOP,[Establish a primary point of contact for the ...
3,Quality Assurance SOP,[Review all deliverables for accuracy and comp...
4,Billing SOP,[Issue invoices according to the terms agreed ...
5,Project Closure SOP,[Once all deliverables have been submitted and...
6,Confidentiality SOP,[All client information should be treated as c...


In [10]:
#Training Data
SOPs = parsed_sops
Company_Profile = pd.concat([parsed_company_profile, parsed_company_profile_1])
Job_Descriptions = pd.concat([parsed_job_descriptions, parsed_job_descriptions_1])
Project_Documentation =  pd.concat([parsed_project_documentation, parsed_system_documentation])



In [29]:
SOPs

,Title,Steps,Steps_processed,Title_processed
0,Client Acquisition SOP,[Identify potential clients through market res...,identify potential client market research refe...,client acquisition sop
1,Project Planning SOP,"[Once a client contract is signed, assign a pr...",client contract signed assign project manager ...,project planning sop
2,Client Communication SOP,[Establish a primary point of contact for the ...,establish primary point contact client. regula...,client communication sop
3,Quality Assurance SOP,[Review all deliverables for accuracy and comp...,review deliverable accuracy completeness sendi...,quality assurance sop
4,Billing SOP,[Issue invoices according to the terms agreed ...,issue invoice according term agreed contract. ...,billing sop
5,Project Closure SOP,[Once all deliverables have been submitted and...,deliverable submitted approved client close pr...,project closure sop
6,Confidentiality SOP,[All client information should be treated as c...,client information treated confidential shared...,confidentiality sop


In [15]:
Company_Profile

,Category,Information
0,Industry,Software Development & IT Services
1,Location,"San Francisco, California, USA"
2,Founded,2007
3,Number of Employees,500-1000
4,CEO,Jane Smith
5,Description,Blue Ocean Technologies is a leading software ...
0,Industry,Renewable Energy
1,Location,"Austin, Texas, USA"
2,Founded,2010
3,Number of Employees,200-500


In [28]:
Job_Descriptions

,Position,Responsibilities,Requirements,Responsibilities_processed,Requirements_processed,Position_processed
0,Project Manager,Define and confirm project scope. Develop and ...,Proven work experience as a project manager in...,define confirm project scope develop manage de...,proven work experience project manager sector ...,project manager
1,Lead AI Specialist,Lead the development and deployment of AI mode...,Proven experience as an AI Specialist or simil...,lead development deployment ai model collabora...,proven experience ai specialist similar role u...,lead ai specialist
2,AI Specialists,Assist in developing and deploying AI models. ...,Proven experience as an AI Specialist or simil...,assist developing deploying ai model work cros...,proven experience ai specialist similar role u...,ai specialist
3,Software Developer,Develop software solutions by studying informa...,Proven work experience as a Software Engineer ...,develop software solution studying information...,proven work experience software engineer softw...,software developer
4,QA Tester,"view requirements, specifications, and technic...",Proven work experience in software development...,view requirement specification technical desig...,proven work experience software development pr...,qa tester
0,Project Manager,Define and confirm project scope. Develop and ...,Proven work experience as a project manager in...,define confirm project scope develop manage de...,proven work experience project manager sector ...,project manager
1,Lead Mobile Application Developer,Design and build advanced applications for the...,Proven work experience as a Mobile developer. ...,design build advanced application android io p...,proven work experience mobile developer demons...,lead mobile application developer
2,Mobile Application Developers,Work on bug fixing and improving application p...,Proven work experience as a Mobile developer. ...,work bug fixing improving application performa...,proven work experience mobile developer demons...,mobile application developer
3,UI/UX Designer,"Design and deliver wireframes, user stories, u...",Three or more years of UX design experience. P...,design deliver wireframes user story user jour...,three year ux design experience preference giv...,ui ux designer
4,Quality Assurance Tester,"view requirements, specifications, and technic...",Proven work experience in software development...,view requirement specification technical desig...,proven work experience software development pr...,quality assurance tester


In [25]:
Project_Documentation

,Section,Content,Content_processed,Section_processed
0,# Project Documentation for Blue Ocean Technol...,1. Introduction,introduction,project documentation blue ocean technology ai...
1,This document outlines the project plan for de...,2. Project Overview,project overview,document outline project plan developing ai dr...
2,"The project involves the design, development, ...",3. Technologies in Use,technology use,project involves design development implementa...
3,"- **AI and Machine Learning**: Python, with it...",4. Project Scope,project scope,ai machine learning python vast array library ...
4,,# 4.1 In-Scope,scope,
5,"- Designing, developing, and implementing the ...",# 4.2 Out-of-Scope,scope,designing developing implementing ai driven so...
6,- Support for the software beyond one year aft...,5. Stakeholders,stakeholder,support software beyond one year implementatio...
7,- Project Manager: Responsible for project ove...,6. Project Timeline,project timeline,project manager responsible project oversight ...
8,The project is expected to be completed in 9 m...,7. Project Deliverables,project deliverable,project expected completed month following mil...
9,- AI-driven software solution. \n- User manua...,8. Risk Management,risk management,ai driven software solution user manual traini...


In [11]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [12]:
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove numbers and punctuation
    text = re.sub(r'[\d\W]+', ' ', text)
    # Tokenize into words
    tokens = word_tokenize(text)
    # Remove stopwords and lemmatize
    processed_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # Re-join tokens into a string
    processed_text = ' '.join(processed_tokens)
    return processed_text

In [20]:
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove numbers and punctuation
    text = re.sub(r'[\d\W]+', ' ', text)
    # Tokenize into words
    tokens = word_tokenize(text)
    # Remove stopwords and lemmatize
    processed_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # Re-join tokens into a string
    processed_text = ' '.join(processed_tokens)
    return processed_text

def preprocess_list(steps_list):
    # Process each step in the list
    processed_list = [preprocess_text(step) for step in steps_list]
    # Combine processed steps into a single string (optional)
    combined_processed_text = '. '.join(processed_list)
    return combined_processed_text


In [27]:
# Preprocess SOPs (assuming the relevant text is in the 'Steps' column)
SOPs['Steps_processed'] = SOPs['Steps'].apply(preprocess_list)
SOPs['Title_processed'] = SOPs['Title'].apply(preprocess_text)


# Preprocess Job Descriptions (assuming the relevant text is in the 'Responsibilities' and 'Requirements' columns)
Job_Descriptions['Responsibilities_processed'] = Job_Descriptions['Responsibilities'].apply(preprocess_text)
Job_Descriptions['Requirements_processed'] = Job_Descriptions['Requirements'].apply(preprocess_text)
Job_Descriptions['Position_processed'] = Job_Descriptions['Position'].apply(preprocess_text)
# Preprocess Project Documentation (assuming the relevant text is in the 'Content' column)
Project_Documentation['Content_processed'] = Project_Documentation['Content'].apply(preprocess_text)
Project_Documentation['Section_processed'] = Project_Documentation['Section'].apply(preprocess_text)


In [31]:
import spacy
from collections import Counter
import pandas as pd

# Load the language model
nlp = spacy.load("en_core_web_sm")

def extract_keywords(text):
    doc = nlp(text)
    keywords = [chunk.text for chunk in doc.noun_chunks]
    keyword_freq = Counter(keywords)
    return keyword_freq.most_common(10)

def aggregate_text_from_column(df, column_name):
    # Combine all text entries into a single string
    aggregated_text = ' '.join(df[column_name].astype(str))
    return aggregated_text

# Aggregate text and extract keywords for each DataFrame
sops_aggregated_text = aggregate_text_from_column(SOPs, 'Steps')
keywords_sops = extract_keywords(sops_aggregated_text)

job_descriptions_aggregated_text = aggregate_text_from_column(Job_Descriptions, 'Responsibilities')  # assuming 'Responsibilities' column
keywords_jobdescription = extract_keywords(job_descriptions_aggregated_text)

project_documentation_aggregated_text = aggregate_text_from_column(Project_Documentation, 'Content')  # assuming 'Content' column
keywords_projectdocumentation = extract_keywords(project_documentation_aggregated_text)


In [32]:
# Check for null values in each column
null_values = Project_Documentation.isnull().sum()

# Display the count of null values per column
print(null_values)

Section              0
Content              0
Content_processed    0
Section_processed    0
dtype: int64


In [43]:
# Assuming job_descriptions and job_requirements are lists or Series of text data
tfidf = TfidfVectorizer()

# Combine roles and requirements for TF-IDF
combined_text = Job_Descriptions['Position_processed'] + Job_Descriptions['Requirements_processed']
tfidf_matrix = tfidf.fit_transform(combined_text)

# Split the matrix back into roles and requirements
midpoint = len(Job_Descriptions)
roles_tfidf = tfidf_matrix[:midpoint]
requirements_tfidf = tfidf_matrix[midpoint:]

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize the vectorizer
tfidf = TfidfVectorizer()

# Vectorize Responsibilities and Requirements separately
roles_tfidf = tfidf.fit_transform(Job_Descriptions['Position_processed'])
requirements_tfidf = tfidf.transform(Job_Descriptions['Requirements_processed'])

# Calculate cosine similarity
# Note: This compares each role's responsibilities against its requirements
similarities = cosine_similarity(roles_tfidf, requirements_tfidf)

# To get a sense of the similarity for each role
for i, role in enumerate(Job_Descriptions['Position']):
    print(f"Role: {role}, Similarity: {similarities[i][i]}")


Role: Project Manager, Similarity: 0.7688003655972753
Role: Lead AI Specialist, Similarity: 0.8164965809277261
Role: AI Specialists, Similarity: 1.0000000000000002
Role: Software Developer, Similarity: 0.8894049548188798
Role: QA Tester, Similarity: 0.2199430190411493
Role: Project Manager, Similarity: 0.7688003655972753
Role: Lead Mobile Application Developer, Similarity: 0.8569861438557037
Role: Mobile Application Developers, Similarity: 1.0
Role: UI/UX Designer, Similarity: 0.5163977794943223
Role: Quality Assurance Tester, Similarity: 0.34989927321161446


In [46]:
# Initialize the matrix with zeros
matrix = pd.DataFrame(0, index=Job_Descriptions['Position'], columns=Job_Descriptions['Requirements_processed'])

# Populate the matrix with cosine similarity scores
# Assuming 'similarities' is the cosine similarity matrix you computed earlier
for i, role in enumerate(Job_Descriptions['Position']):
    for j, requirement in enumerate(Job_Descriptions['Requirements_processed']):
        matrix.at[role, requirement] = similarities[i][j]

# Display the matrix
print(matrix)

Requirements_processed             proven work experience project manager sector solid technical background understanding hand experience software development web technology excellent client facing internal communication skill excellent written verbal communication skill strong working knowledge microsoft office  \
Position                                                                                                                                                                                                                                                                                                                    
Project Manager                                                             0.768800                                                                                                                                                                                                                                        
Lead AI Specialist                               

The matrix created is a comprehensive representation of the alignment between job roles and requirements based on cosine similarity. It essentially quantifies how closely the language used in each job role's description matches the language in each requirement. This matrix can be a valuable tool for understanding the relevance of specific requirements to different job roles, and subsequently, for tailoring learning or training programs for new hires in these roles.

In [67]:
new_hire_role = 'project manager'
if new_hire_role in  Job_Descriptions['Position_processed'].values:
  print(new_hire_role)
# Job_Descriptions['Position_processed'].values



project manager


In [86]:
jobs_available = Job_Descriptions['Position_processed'].unique().tolist()
job_role = 'Mobile Application Developers'
# print(job_role.lower())
for i in jobs_available:
  if i == job_role.lower():
    print(Job_Descriptions[Job_Descriptions['Position_processed'] == i])




In [89]:
def recommend_learning_materials(job_role, Job_Descriptions, matrix):
    # Convert both job_role and DataFrame entries to lowercase for comparison
    """
    Recommends learning materials based on the job role.
    Args:
    - job_role (str): The job role of the new hire.
    - job_descriptions_df (pd.DataFrame): DataFrame containing job descriptions.
    - matrix (pd.DataFrame): DataFrame containing cosine similarity scores.

    Returns:
    - list: A list of recommended learning materials.
    """
    # Check if the job role exists in the DataFrame
    if job_role in Job_Descriptions['Position'].values:
        role_index = Job_Descriptions[Job_Descriptions['Position'] == job_role].index[0]
        print("Job Role")
        # Retrieve the similarity scores for the given role
        role_similarities = matrix.iloc[role_index]

        # Sort the scores and get the top 5 requirements
        top_requirements_indices = role_similarities.argsort()[-5:][::-1]
        top_requirements = Job_Descriptions['Requirements_processed'].iloc[top_requirements_indices]

        return top_requirements.tolist()
    else:
        # Return an empty list if the role is not found
        return []

# Example usage
job_role = 'Project Manager'
recommended_materials = recommend_learning_materials(job_role, Job_Descriptions, matrix)

# Output the recommended learning materials
if recommended_materials:
    print(f"Recommended Learning Materials for {job_role}: {recommended_materials}")
else:
    print(f"No recommendations found for the role: {job_role}")


Job Role
Recommended Learning Materials for Project Manager: ['proven work experience project manager sector solid technical background understanding hand experience software development web technology excellent client facing internal communication skill excellent written verbal communication skill strong working knowledge microsoft office', 'proven work experience project manager sector solid technical background understanding hand experience software development web technology excellent client facing internal communication skill excellent written verbal communication skill strong working knowledge microsoft office', 'proven work experience software development proven work experience software quality assurance strong knowledge software qa methodology tool process experience writing clear concise comprehensive test plan test case', 'three year ux design experience preference given candidate experience designing complex solution complete digital environment expertise standard ux softwar

In [93]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [96]:
path = '/content/drive/My Drive/MyData/'

In [97]:
# Save the matrix DataFrame
with open('matrix.pkl', 'wb') as file:
    pickle.dump(matrix, file)

# Save the Job_Descriptions DataFrame
with open('job_descriptions.pkl', 'wb') as file:
    pickle.dump(Job_Descriptions, file)


In [48]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def recommend_learning_materials(Job_Descriptions['Position_processed'], Job_Descriptions, matrix):
    """
    Recommends learning materials based on the job role.
    Args:
    - job_role (str): The job role of the new hire.
    - job_descriptions_df (pd.DataFrame): DataFrame containing job descriptions.
    - matrix (pd.DataFrame): DataFrame containing cosine similarity scores.

    Returns:
    - list: A list of recommended learning materials.
    """
    # Find the index of the job role in the job descriptions dataframe
    role_index = Job_Descriptions[Job_Descriptions['Position_processed'] == job_role].index[0]

    # Retrieve the similarity scores for the given role
    role_similarities = matrix.iloc[role_index]

    # Sort the scores and get the top 5 requirements
    top_requirements_indices = role_similarities.argsort()[-5:][::-1]
    top_requirements = Job_Descriptions['Requirements_processed'].iloc[top_requirements_indices]

    return top_requirements.tolist()

# Example: Integration with the onboarding system
# Assuming 'new_hire_role' is the role assigned to a new hire
new_hire_role = 'Software Developer'
recommended_materials = recommend_learning_materials(new_hire_role, Job_Descriptions, matrix)

# Output the recommended learning materials
print(f"Recommended Learning Materials for {new_hire_role}: {recommended_materials}")


SyntaxError: ignored

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Let's assume 'job_descriptions_processed' is a DataFrame with your processed job descriptions
tfidf = TfidfVectorizer(max_features=500)  # You can adjust 'max_features' based on your dataset
features = tfidf.fit_transform(Job_Descriptions['Responsibilities_processed'])


In [38]:
similarities = cosine_similarity(features, )
top_matches_indices = similarities.argsort()[0][-5:]  # Get indices of top 5 similar materials


TypeError: ignored

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

# Assuming 'learning_materials_features' is a matrix of features for each learning material
# and 'job_role_features' is the feature vector for a specific job role
similarities = cosine_similarity(job_role_features, learning_materials_features)
top_matches = similarities.argsort()[0][-5:]  # Get the indices of top 5 similar materials


NameError: ignored

In [51]:
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [ ]:


# Let's assume `documents` is a list of text data, and `labels` are the corresponding categories
documents = [sops_processed, job_descriptions_processed, project_documentation_processed, ...]  # add all documents
labels = [...]  # your list of labels corresponding to each document

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(documents, labels, test_size=0.2, random_state=42)
